In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 10000

ip = np.random.randint(0, 4, N)  # Random QPSK symbols [0, 1, 2, 3]
s = np.exp(1j * ip * np.pi / 2)  # QPSK modulation

eb_no_db = np.arange(0, 25)

bit_error_rate = np.zeros(len(eb_no_db))

def Zero_Forcing_QPSK():
    for jj in range(len(eb_no_db)):
        nErr = 0
        received_bits_list = []

        for ii in range(N // 2):
            h11 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            h12 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            h21 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            h22 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            n1 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())
            n2 = (1 / np.sqrt(2)) * (np.random.rand() + 1j * np.random.rand())

            x1 = s[2 * ii]
            x2 = s[2 * ii + 1]

            H = np.array([[h11, h12], [h21, h22]])
            n = np.array([[n1], [n2]])
            x = np.array([[x1], [x2]])

            Hx = np.dot(H, x)
            y = Hx + n * 10 ** (-eb_no_db[jj] / 20)

            # Forming the Zero Forcing equalization matrix W = inv(H^H*H)*H^H
            H_Hermitian = np.conj(H.T)
            H_Hermitian_H = np.dot(H_Hermitian, H)
            condition_number = np.linalg.cond(H_Hermitian_H)
            H_Hermitian_H_inverse = np.linalg.pinv(H_Hermitian_H)
            W = np.dot(H_Hermitian_H_inverse, H_Hermitian)
            x_hat = np.dot(W, y)
            
            # Demodulate QPSK symbols from received symbols
            received_symbols = np.angle(x_hat) / (np.pi / 2)
            received_bits = np.floor((received_symbols + 0.5) % 4)
            
            # Convert QPSK symbols to bits
            received_bits_list.extend(np.unpackbits(received_bits.astype(np.uint8)))

        # Convert the original QPSK symbols to bits
        original_bits_list = np.unpackbits(ip.astype(np.uint8))

        nErr = np.sum(original_bits_list != received_bits_list)
        bit_error_rate[jj] = nErr / (N * 2)  # Multiply by 2 since QPSK has 2 bits per symbol

    return bit_error_rate

result_qpsk = Zero_Forcing_QPSK()

plt.figure()
plt.plot(eb_no_db, result_qpsk, 'mo-', linewidth=2, label='nTx=2 nRx=2 (zero_forcing)')
plt.grid(True)
plt.legend()
plt.xlabel('Eb/No, dB')
plt.ylabel('Bit Error Rate')
plt.title('BER for QPSK modulation with zero_forcing_algorithm')
plt.show()